<a href="https://colab.research.google.com/github/JMindpalace/Machine_Learning/blob/main/Example/003.%20%EB%8B%B9%EB%87%A8(%EC%A3%BC%EC%82%AC%26%EA%B5%AC%EA%B0%95%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문제 정의
> 당뇨 치료제 중 경구복용 인슐린제의 효능 확인

In [1]:
import pandas as pd
import numpy as np
from google.colab import files

# Read Data-set

In [2]:
files.upload()
patients = pd.read_csv('patients_info.csv')

Saving patients_info.csv to patients_info (1).csv


In [3]:
files.upload()
insuline_test = pd.read_csv('insuline_test.csv')

Saving insuline_test.csv to insuline_test (1).csv


In [4]:
files.upload()
side_effects = pd.read_csv('side_effects.csv')

Saving side_effects.csv to side_effects (1).csv


# Columns

#### **`patients` 데이터**  
> - 환자고유번호: 테이블의 기본키
- **이름: 환자 이름**
- 나이: 환자의 성별과 나이
- 환자식별번호: 시스템에 저장되어 있는 5자리 숫자로 구성된 환자 번호
- 키: 환자의 키
- 몸무게: 환자의 몸무게
- 체질량지수: 환자의 체질량지수
- ( 체질량지수 = (몸무게 (kg) / (키 (cm) x 키 (cm))) x 10,000 )
- 전화번호: 환자의 개인 전화번호 
<br>

#### **`insuline test` 데이터** 
> - **이름: 환자 이름**
- 경구용_인슐린: 주사제 인슐린의 일일 투여량의 중앙값(median) 에서 24주 뒤의, 경구 인슐린의 일일 투여량 중앙값 <br>
  - `주사제 인슐린의 일일 투여량 중앙값`**u** - `24주 뒤, 경구 인슐린의 일일 투여량의 중앙값`**u**  
  - **u**는 인슐린의 측정 단위 
- 주사제_인슐린: 주사제 인슐린의 일일 투여량의 중앙값에서 24주 뒤의, 주사제 인슐린의 일일 투여량의 중앙값 <br>
  - `주사제 인슐린의 일일 투여량의 중앙값`**u** - `24주 뒤, 주사제 인슐린의 일일 투여량의 중앙값`**u**  
- 혈당수치_시작: 첫번째 주에 환자의 혈당수치 레벨(%)  
- 혈당수치_끝: 24주 동안의 실험 시행 후, 마지막 주 환자의 혈당수치 레벨 (%)
- 혈당수치_변화: 환자의 혈당수치 레벨의 변화 (%) 
  - `혈당수치_시작 - 혈당수치_끝` 
<br>

#### **`side_effects` 데이터** 
> - **이름: 환자 이름**
- 부작용: 환자의 의해서 보고된 부작용 

# Target
#### 치료제 효능 검증: 주사제와 경구제의 혈당 수치 변화
- 혈당 수치의 0.4정도의 수치가 낮아졌다면 성공

  - 최초 350명 전원 주사제 투여로 기본 혈당수치 측정
  - 4주뒤 대조군 175명 주사, 실험군 175명 경구제 투여
  - 24주 후 각 약물에 대한 혈당수치 측정

In [5]:
assert patients.shape == (509, 8)
assert insuline_test.shape == (350, 6)
assert side_effects.shape == (35, 2)

# 데이터 전처리

In [6]:
patients_clean = patients.copy()
insuline_test_clean = insuline_test.copy()
side_effects_clean = side_effects.copy()

## 품질의 문제

### patients_clean

In [7]:
# 환자이름 에러(_ ! . 숫자 등 포함)
import re
def has_errors(inputstring):
  return bool( re.search('[_, !, ., \d+]', inputstring) )

# 본문 적용 전 test - error_names
# error_names = patients_clean[ patients_clean.이름.apply(has_errors) ]
# error_names.이름 = error_names.이름.str.replace('[_,!,.,\d+]', '')
patients_clean.이름 = patients_clean.이름.str.replace('[_, !, ., \d+]', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


In [8]:
# 나이 칼럼 - 성별(한글로 변환)과 나이 분리 및 타입 변환
patients_clean['환자나이'] = patients_clean.나이.str.extract('(\d+)').astype('int')
patients_clean['성별'] = patients_clean.나이.str.replace(r'[^a-zA-Z]', '', regex=True)

patients_clean = patients_clean.drop('나이', axis=1)

patients_clean['성별'] = patients_clean.성별.replace( {'male':'남' , 'female':'여' } )
patients_clean['성별'] = patients_clean.성별.astype('category')

In [9]:
# 환자식별번호 - 실수 -> object로 변환(비교가 아닌 고유값), 5자리로 통일, 모르는 값은 nan으로 채우기
patients_clean.환자식별번호 = patients_clean.환자식별번호.astype(str).str[:-2].str.pad(5, fillchar='0')
patients_clean.환자식별번호 = patients_clean.환자식별번호.replace( '0000n', np.nan )

In [10]:
# 전화번호 포맷(010-xxxx-xxxx) 일관되게 맞추기
def convert(phone_num):
  phone_num = re.sub(r'\D', '', phone_num)[-8:]
  final_num = '010' + '-' + phone_num[:4] + '-' + phone_num[-4:]
  return final_num

patients_clean.전화번호 = patients_clean.전화번호.apply(convert)

In [11]:
# 키, 몸무게, BMI - Missing value
import math
# bool_series = (patients_clean.키.isnull() | patients_clean.몸무게.isnull()| patients_clean.체질량지수.isnull())
# missing_in_three = patients_clean[bool_series] 
# insert_values(missing_in_three) -- 데이터 적용 전 test

def insert_values(df):
  for i, height in enumerate(df.키):
    if math.isnan(height):
      df.iloc[i,3] = round( (10000 * df.iloc[i,4] / df.iloc[i,5]) **(1/2) , 2 )
      
  for i, weight in enumerate(df.몸무게):
    if math.isnan(weight):
      df.iloc[i,4] = round((df.iloc[i,5]/10000)*(df.iloc[i,3])**2,2)

  for i, bmi in enumerate(df.체질량지수):
    if math.isnan(bmi):
      df.iloc[i,5] = round(10000*df.iloc[i,3]/(df.iloc[i,4])**2,2)
  
  return df
patients_clean = insert_values(patients_clean)

# 키, 몸무게, BMI - 부정확한 데이터 처리(수학적 관계로 재계산)
def fix_wrong_values(df):
  for i, weight in enumerate(df.몸무게):
    if weight < 10:
      df.iloc[i,4] = round((df.iloc[i,5]/10000)*(df.iloc[i,3])**2,2)

  for i, height in enumerate(df.키):
      if height < 100:
        df.iloc[i, 3] = round((10000*(df.iloc[i,4]/df.iloc[i,5]))**(1/2),2)

  for i, bmi in enumerate(df.체질량지수):
    if bmi < 10 or bmi > 100:
      df.iloc[i, 5] = round((df.iloc[i, 4] / (df.iloc[i,3] * df.iloc[i,3]))*10000,2)

  return df
new_bmi = fix_wrong_values(patients_clean).체질량지수
(patients_clean['체질량지수'] == new_bmi).all()

True

In [12]:
# 중복값 처리 - 동명이인을 제외한 이름과 전화번호가 동일한 환자를 제거
patients_clean = patients_clean.drop_duplicates(subset=['이름', '전화번호'])

### insuline_test_clean

#### 품질의 문제

In [13]:
# 핵심 Target 혈당수치변화 Missing value 조정
insuline_test_clean.혈당수치_변화 = (insuline_test_clean.혈당수치_시작 - insuline_test_clean.혈당수치_끝)

#### 구조적 문제

In [15]:
# 경구용, 주사제 인슐린 2칸에 시작과 끝 데이터가 혼합됨 -> 분리
insuline_test_clean = pd.melt(
    insuline_test_clean, id_vars = ['이름', '혈당수치_시작', '혈당수치_끝', '혈당수치_변화'],
    var_name = '인슐린_종류', value_name='투여량' )

insuline_test_clean = insuline_test_clean[ insuline_test_clean.투여량 != '-' ] # -로 표시된 Missing value 제외

insuline_test_clean['시작_투여량'], insuline_test_clean['마지막_투여량'] = insuline_test_clean[
    '투여량'].str.split(' - ', 1).str
    
insuline_test_clean = insuline_test_clean.drop('투여량', axis=1)
insuline_test_clean.인슐린_종류 = insuline_test_clean.인슐린_종류.str.replace('_', ' ')

# 투여량 2개 모두 u를 제거하고 데이터 타입 변경
insuline_test_clean.시작_투여량 = insuline_test_clean.시작_투여량.str.strip('u').astype(int)
insuline_test_clean.마지막_투여량 = insuline_test_clean.마지막_투여량.str.strip('u').astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


### side_effects_clean

In [18]:
# 부작용 정보만 insuline_test_clean에 병합
insuline_test_clean = side_effects_clean.merge(insuline_test_clean, how='left', on = ['이름'])

# insuline_test_clean도 tidy가 되어 키 값이 필요함 -> patients_clean의 환자고유번호로 병합
primary_table = patients[['환자고유번호', '이름']]
insuline_test_clean = pd.merge(insuline_test_clean, primary_table, how='left', on=['이름'])
insuline_test_clean = insuline_test_clean.drop(['이름'], axis=1)

In [ ]:
col_order = ['환자고유번호', '인슐린_종류', '시작_투여량', '마지막_투여량', '혈당수치_시작', '혈당수치_끝', '혈당수치_변화', '부작용']
insuline_test_clean = insuline_test_clean[col_order]

In [ ]:
patients_clean.to_csv('patients_clean.csv', index=False)
insuline_test_clean.to_csv('insuline_test_clean.csv', index=False)